## 한국어 욕설 데이터 추출

- 데이터 원본 : LLM 학습용 데이터 내 유해표현 검출 AI모델 학습용 데이터
- https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=71833


In [2]:
# KoNLPy 및 JPype 설치 Java 설치 (Amazon Linux 기반)
!sudo yum install -y java-1.8.0-openjdk-devel
!pip install konlpy
!pip install JPype1==1.3.0  # 최신 버전이 호환성 문제가 있을 수 있어 안정 버전 지정

Loaded plugins: dkms-build-requires, extras_suggestions, kernel-livepatch,
              : langpacks, priorities, update-motd, versionlock
amzn2-core                                               | 3.6 kB     00:00     
amzn2extra-docker                                        | 2.9 kB     00:00     
amzn2extra-kernel-5.10                                   | 3.0 kB     00:00     
amzn2extra-livepatch                                     | 2.9 kB     00:00     
amzn2extra-lustre                                        | 2.5 kB     00:00     
amzn2extra-python3.8                                     | 2.9 kB     00:00     
centos-extras                                            | 2.9 kB     00:00     
copr:copr.fedorainfracloud.org:vbatts:shadow-utils-newxi | 3.3 kB     00:00     
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
nvidia-container-toolkit/x86_64/signature                |  833 B     00:00

### 유해 카테고리 분류 데이터 로드

In [7]:
import pandas as pd
import re

#stage2_data.csv 도 로드하는 코드
stage2_data = pd.read_csv('data/2.harmful_category_classification/stage2_data.csv', index_col=0)

print("stage1_data의 결측치 확인")
stage2_data.isnull().sum() #결측치 확인



stage1_data의 결측치 확인


모욕           0
욕설           0
외설           0
폭력위협/범죄조장    0
성혐오          0
연령           0
인종/지역        0
장애           0
종교           0
정치성향         0
직업           0
정상           0
dtype: int64

### 추출용 유틸리티함수

In [8]:
# 따옴표 정리 함수
def clean_quotes(text):
    if isinstance(text, str):
        # 여러 중첩된 따옴표를 하나의 따옴표로 정리
        cleaned_text = re.sub(r"'{2,}", "'", text)  # 작은따옴표 정리
        cleaned_text = re.sub(r'"{2,}', '"', cleaned_text)  # 큰따옴표 정리
        
        # 필요하다면 따옴표를 완전히 제거
        # cleaned_text = re.sub(r"['\"]", "", cleaned_text)
        
        return cleaned_text
    return text


def remove_all_quotes(text):
    """
    텍스트에서 모든 종류의 따옴표를 제거하는 함수
    
    Parameters:
    text (str): 따옴표를 제거할 원본 텍스트
    
    Returns:
    str: 따옴표가 제거된 텍스트
    """
    if not isinstance(text, str):
        return text
    
    # 작은따옴표('), 큰따옴표(") 모두 제거
    cleaned_text = re.sub(r"['\"]", "", text)
    
    return cleaned_text

# DataFrame에 적용하는 예시
# stage2_data['문장_정제'] = stage2_data['문장'].apply(remove_all_quotes)


def remove_quotes_and_special_chars(text):
    """
    텍스트에서 따옴표와 특수문자를 제거하는 함수
    
    Parameters:
    text (str): 처리할 원본 텍스트
    
    Returns:
    str: 따옴표와 특수문자가 제거된 텍스트
    """
    if not isinstance(text, str):
        return text
    
    # 1. 따옴표(', ") 제거
    text = re.sub(r"['\"]", "", text)
    
    # 2. 백틱(`) 제거
    text = re.sub(r"`", "", text)
    
    # 3. 기타 일반적인 특수문자 제거 (필요에 따라 조정 가능)
    # 여기서는 백틱, 역따옴표, 중괄호, 대괄호, 작은/큰 괄호 등을 제거
    text = re.sub(r"[`´''""『』\[\](){}]", "", text)
    
    return text

# 확장 버전 - 더 많은 특수문자 제거가 필요한 경우
def remove_all_special_chars(text, keep_korean=True, keep_english=True, 
                             keep_numbers=True, keep_spaces=True):
    """
    텍스트에서 선택적으로 특수문자를 제거하는 확장 함수
    
    Parameters:
    text (str): 처리할 원본 텍스트
    keep_korean (bool): 한글 유지 여부
    keep_english (bool): 영어 유지 여부
    keep_numbers (bool): 숫자 유지 여부
    keep_spaces (bool): 공백 유지 여부
    
    Returns:
    str: 특수문자가 제거된 텍스트
    """
    if not isinstance(text, str):
        return text
    
    pattern = ""
    
    # 유지할 문자 패턴 설정
    if keep_korean:
        pattern += "가-힣"
    if keep_english:
        pattern += "a-zA-Z"
    if keep_numbers:
        pattern += "0-9"
    if keep_spaces:
        pattern += " "
    
    # 지정된 문자 외 모든 것 제거
    return re.sub(f"[^{pattern}]", "", text)
    
def extract_quoted_words(text):
    """
    텍스트에서 따옴표(', ", `, 작은/큰따옴표 등) 안에 있는 단어나 구문을 추출하는 함수
    
    Parameters:
    text (str): 처리할 원본 텍스트
    
    Returns:
    list: 따옴표 안에 있던 단어/구문 목록
    """
    if not isinstance(text, str):
        return []
    
    # 여러 유형의 따옴표 패턴 정의
    patterns = [
        r"'([^']+)'",         # 작은따옴표: '단어'
        r'"([^"]+)"',         # 큰따옴표: "단어"
        r"`([^`]+)`",         # 백틱: `단어`
        r"''([^']+)''",       # 이중 작은따옴표: ''단어''
        r"'''([^']+)'''",     # 삼중 작은따옴표: '''단어'''
        r"『([^』]+)』",       # 한국어 인용부호: 『단어』
        r"「([^」]+)」"        # 한국어 인용부호: 「단어」
    ]
    
    results = []
    for pattern in patterns:
        matches = re.findall(pattern, text)
        results.extend(matches)
    
    # 중복 제거 및 공백 제거
    results = [word.strip() for word in results]
    results = list(set(results))  # 중복 제거
    
    # AWS Guardrail 제한사항: 최대 3단어까지만 허용
    results = [word for word in results if len(word.split()) <= 3]
    
    return results

# DataFrame에 적용하여 차단 단어 목록 생성
def create_block_list_from_dataframe(df, text_column):
    """
    DataFrame의 텍스트 열에서 따옴표 안의 단어를 추출하여 차단 목록 생성
    
    Parameters:
    df (DataFrame): 텍스트 데이터가 있는 DataFrame
    text_column (str): 텍스트가 있는 열 이름
    
    Returns:
    DataFrame: 차단 단어 목록이 있는 DataFrame
    """
    all_words = []
    
    # 각 텍스트에서 따옴표 안의 단어 추출
    for text in df[text_column]:
        extracted_words = extract_quoted_words(text)
        all_words.extend(extracted_words)
    
    # 중복 제거 및 정렬
    all_words = list(set(all_words))
    all_words.sort()
    
    # 차단 목록 DataFrame 생성 (AWS Guardrail 형식에 맞게)
    block_list_df = pd.DataFrame(all_words, columns=['blocked_words'])
    
    return block_list_df

import pandas as pd
import re
from difflib import SequenceMatcher

# 기존 함수는 이미 중복 제거 기능이 포함되어 있지만, 더 강화된 중복 처리 함수를 작성합니다
def find_and_merge_duplicates(block_list_df):
    """
    차단 목록에서 중복 또는 유사 단어를 찾아 병합하는 함수
    
    Parameters:
    block_list_df (DataFrame): 차단 단어 목록이 있는 DataFrame
    
    Returns:
    DataFrame: 중복이 제거된 차단 단어 목록 DataFrame
    """
    # 1. 기본 전처리
    words = block_list_df['blocked_words'].tolist()
    processed_words = []
    
    for word in words:
        # 공백 표준화 (여러 공백을 하나로)
        word = re.sub(r'\s+', ' ', word).strip()
        processed_words.append(word)
    
    # 2. 정확한 중복 제거 (대소문자 무시)
    unique_words = []
    unique_words_lower = []
    
    for word in processed_words:
        word_lower = word.lower()
        if word_lower not in unique_words_lower:
            unique_words.append(word)
            unique_words_lower.append(word_lower)
    
    # 3. 유사 단어 찾기 및 병합 (선택 사항)
    similar_groups = []
    used_indices = set()
    
    # 유사도 기준 (0.8은 80% 유사함을 의미)
    similarity_threshold = 0.8
    
    for i in range(len(unique_words)):
        if i in used_indices:
            continue
            
        group = [unique_words[i]]
        used_indices.add(i)
        
        for j in range(i+1, len(unique_words)):
            if j in used_indices:
                continue
                
            # 유사도 계산 (Levenshtein 거리 기반)
            similarity = SequenceMatcher(None, 
                                         unique_words[i].lower(), 
                                         unique_words[j].lower()).ratio()
            
            if similarity >= similarity_threshold:
                group.append(unique_words[j])
                used_indices.add(j)
        
        if len(group) > 1:  # 유사한 단어가 발견된 경우
            similar_groups.append(group)
    
    # 4. 최종 결과 생성
    final_words = []
    
    # 유사 그룹에 포함되지 않은 단어 추가
    for i, word in enumerate(unique_words):
        found_in_group = False
        for group in similar_groups:
            if word in group:
                found_in_group = True
                break
        
        if not found_in_group:
            final_words.append(word)
    
    # 유사 그룹의 대표 단어 추가 (가장 짧은 단어 선택)
    for group in similar_groups:
        # 가장 짧은 단어를 대표로 선택 (또는 다른 기준 적용 가능)
        representative = min(group, key=len)
        final_words.append(representative)
        
        # 유사 단어 그룹 출력 (참고용)
        print(f"유사 단어 그룹 병합: {group} -> {representative}")
    
    # 최종 DataFrame 생성
    final_df = pd.DataFrame(final_words, columns=['blocked_words'])
    
    # 정렬 적용
    final_df = final_df.sort_values('blocked_words').reset_index(drop=True)
    
    return final_df

def filter_common_words(block_list_df):
    """
    차단 목록에서 일반적인 단어/조사/접속사 등을 제거하는 함수
    
    Parameters:
    block_list_df (DataFrame): 차단 단어 목록이 있는 DataFrame
    
    Returns:
    DataFrame: 일반 단어가 제거된 차단 단어 목록 DataFrame
    """
    # 한국어 불용어 목록 (필요에 따라 확장)
    korean_stopwords = [
        # 일반 접속사/조사
        '및', '등', '과', '와', '의', '에', '에서', '로', '으로', '에게', '께', '한', '을', '를', '이', '가',
        
        # 일반 단어
        '것', '수', '듯', '때', '곳', '중', '내', '외', '앞', '뒤', '위', '아래', '좌', '우', 
        
        # 지시어
        '이', '그', '저', '이것', '그것', '저것', '여기', '저기', '거기',
        
        # 시간 관련
        '때', '지금', '현재', '오늘', '내일', '어제', '다음', '이전',
        
        # 숫자 관련
        '하나', '둘', '셋', '넷', '다섯', '첫째', '둘째', '셋째',
        
        # 의문사
        '무엇', '어디', '언제', '누구', '어떤', '어떻게', '왜', '얼마나',
        
        # 기타 일반 단어
        '정도', '경우', '상태', '모두', '모든', '각', '각각', '서로', '우리', '그들', '모든'
    ]
    
    # 너무 짧은 단어 제외 (1글자)
    short_words = [word for word in block_list_df['blocked_words'] if len(word) == 1]
    
    # 불용어 목록과 짧은 단어 목록을 합침
    words_to_exclude = set(korean_stopwords + short_words)
    
    # 필터링 전 개수
    original_count = len(block_list_df)
    
    # 불용어 제외
    filtered_df = block_list_df[~block_list_df['blocked_words'].isin(words_to_exclude)]
    
    # 제외된 단어 목록 출력
    excluded_words = set(block_list_df['blocked_words']) & words_to_exclude
    print(f"제외된 일반 단어/불용어 ({len(excluded_words)}개):")
    print(sorted(list(excluded_words)))
    
    # 필터링 후 개수
    filtered_count = len(filtered_df)
    print(f"\n필터링 전: {original_count}개 단어")
    print(f"필터링 후: {filtered_count}개 단어")
    print(f"제거된 단어 수: {original_count - filtered_count}개")
    
    return filtered_df


import re

def extract_special_quote_pattern(text):
    """
    텍스트에서 ''로 시작하고 '''로 끝나는 패턴 내부의 내용을 추출하는 함수
    
    Parameters:
    text (str): 처리할 원본 텍스트
    
    Returns:
    list: 추출된 내용 목록
    """
    if not isinstance(text, str):
        return []
    
    # ''로 시작하고 '''로 끝나는 패턴을 찾는 정규표현식
    pattern = r"''([^']*)'''"
    
    # 패턴에 맞는 모든 내용 추출
    matches = re.findall(pattern, text)
    
    return matches

def remove_single_char_words(block_list_df):
    """
    차단 목록에서 1글자로 된 단어를 제거하는 함수
    
    Parameters:
    block_list_df (DataFrame): 차단 단어 목록이 있는 DataFrame
    
    Returns:
    DataFrame: 1글자 단어가 제거된 차단 단어 목록 DataFrame
    """
    # 필터링 전 단어 수
    original_count = len(block_list_df)
    
    # 단어 길이 계산 함수 (공백 제외)
    def get_effective_length(word):
        if not isinstance(word, str):
            return 0
        # 공백 제거 후 길이 계산
        return len(word.replace(" ", ""))
    
    # 1글자 단어 찾기
    single_char_words = block_list_df[block_list_df['blocked_words'].apply(get_effective_length) <= 1]
    
    # 1글자 단어 제거하기 (effective length가 2 이상인 단어만 유지)
    filtered_df = block_list_df[block_list_df['blocked_words'].apply(get_effective_length) > 1]
    
    # 필터링 후 단어 수
    filtered_count = len(filtered_df)
    
    # 제거된 단어 출력
    if not single_char_words.empty:
        print(f"제거된 1글자 단어 ({len(single_char_words)}개):")
        #for word in single_char_words['blocked_words'].tolist():
            #print(f"- '{word}'")
    
    print(f"\n필터링 전: {original_count}개 단어")
    print(f"필터링 후: {filtered_count}개 단어")
    print(f"제거된 단어 수: {original_count - filtered_count}개")
    
    return filtered_df


### 칼럼명 확인

In [10]:
print(stage2_data.columns)

Index(['모욕', '욕설', '외설', '폭력위협/범죄조장', '성혐오', '연령', '인종/지역', '장애', '종교', '정치성향',
       '직업', '정상'],
      dtype='object')


In [11]:
print(stage2_data.index)
#2. 만약 인덱스가 문장이라면 인덱스를 열로 변환하기
# 인덱스를 '문장' 열로 변환
stage2_data.reset_index(inplace=True)
stage2_data.rename(columns={'index': '문장'}, inplace=True)

# 이제 문장 열에 접근 가능
print(stage2_data['문장'].head())

Index(['독점은 당연히 막아야지. 이건 사장경제는 경쟁으로 가야 정상순환되는데 독점이 되면 기업이 공산국가처럼 독재하는건데',
       '눈 작다고 한 마디 한 걸로 퇴출시키는 그리스가 훨씬 선진국이네 ㅋㅋㅋ MBC는 사과문 ''끄적'''이고 ''버로우 타'''고 있는데',
       '''도쿄만 똥물'''도 문제지만 해안에 모래는 후쿠시마 흙으로 채웠는데 이선수들 피폭이 걱정된다',
       '''종북''' ''좌파''' ''빨갱이'''들의 마지막 부정선거가 다음 대선이다 . ' 미군 철수 , 국가보안법 폐지 ' 아프간 사태는 우리에게 보내는 마지막 경고 메세지다 .',
       '동료 여경 집단 성희롱’ 태백 경찰관들 12일 징계 논의-----''예비강간마'''''새끼'''들 기사 묻히는 이유 좀 알려주세요',
       '일제시대때 ''친일파'''  ''매국노'''들과 같네 .''토왜'''들도 저리해서 잘먹고 잘 살았겠지?',
       '''쩡개'''꺼눈 특별히 메드인차이나 재품으로 준곳임',
       '빨리 일반 질병으로 분류합시다. 현명한 결정이라고 봅니다. 제약사들 ''비지니스에 놀아나'''지 맙시다.',
       '여기저기 ''정신병 걸린'''애들이 많긴많구나~1950년 전 대한제국인줄 아나?',
       '없던 질문을 자막에 넣은 mbc, 과거 ''광우뻥''' 사태 때도 실체가 없는 것을 진실인양 보도해서 나라를 뒤집었지만 사과 한마디 없음~',
       ...
       ' 따뜻하게 세상을 바라보는 아이처럼, `루카` 속 풍경은 평화롭다',
       '온갖 고문으로 오기섭이 탈진해 있을 때 최달식은 자기 집으로 전화를 걸어 아들과 통화한다',
       '중소·중견기업의 경우 해당 업종의 통계를 입증하기가 쉽지 않고 서비스업의 경우 보조지표중 ‘고용대비 서비스생산 지수 ’외에는 활용할 만한 지표가 없어 기업들이 신청을 주저할 수 있다는 내용이었다',
       '"시 쓰기가 고통스러웠던 적은 없어요

In [12]:
print(stage2_data.columns)

Index(['문장', '모욕', '욕설', '외설', '폭력위협/범죄조장', '성혐오', '연령', '인종/지역', '장애', '종교',
       '정치성향', '직업', '정상'],
      dtype='object')


## DataFrame에 적용하여 차단 목록 생성

In [13]:
# DataFrame에 적용하여 차단 목록 생성
def create_block_list_from_special_quotes(df, text_column):
    """
    DataFrame의 텍스트 열에서 특수 따옴표 패턴 내 단어를 추출하여 차단 목록 생성
    
    Parameters:
    df (DataFrame): 텍스트 데이터가 있는 DataFrame
    text_column (str): 텍스트가 있는 열 이름
    
    Returns:
    DataFrame: 차단 단어 목록이 있는 DataFrame
    """
    all_words = []
    
    # 각 텍스트에서 특수 따옴표 패턴 내 단어 추출
    for text in df[text_column]:
        if isinstance(text, str):
            extracted_words = extract_special_quote_pattern(text)
            all_words.extend(extracted_words)
    
    # 중복 제거 및 정렬
    all_words = list(set(all_words))
    all_words.sort()
    
    # 차단 목록 DataFrame 생성
    block_list_df = pd.DataFrame(all_words, columns=['blocked_words'])
    
    return block_list_df


### 특정 욕설의 값이 있는 카테고리 지정

- 아래 예시는 폭력위협/범죄조장을 사용 

In [20]:
#stage2_data의 칼럼명이 '폭력위협/범죄조장'인 데이터

#정상이 아닌것을 추출
abnormal_threats = stage2_data[stage2_data['정상'] == 0]
abnormal_threats

# 폭력위협/범죄조장 열의 값이 0이 아닌 행만 추출
violence_threats = stage2_data[stage2_data['폭력위협/범죄조장'] > 0]
violence_threats

,문장,모욕,욕설,외설,폭력위협/범죄조장,성혐오,연령,인종/지역,장애,종교,정치성향,직업,정상
38,''Cㅂ ㅏ''' ''ㅇ ㅏㄱ ㅏㅃ ㅏ 뤼''' 다 ''트자 뿔''' ㄹ ㅏ,0,2,0,2,0,0,0,0,0,0,0,0
44,이래서 ''더불어모화당'''은 중국으로 ''추방해'''야..,0,0,0,2,0,0,0,0,0,1,0,0
53,''강간범'''한테 ''폭행'''''강간'''당한 나는 저런모습이엇을까 ㅋㅋ,2,0,0,2,0,0,0,0,0,0,0,0
74,브라질 흑인 **번은 가발쓴 남자가 분명하다.도핑이고 돌핀이고 간에 성별검사부터 해라,2,0,0,1,0,0,0,0,0,0,0,0
79,조선과 명나라 연합군이 일본 정벌하러 가다 태풍만 만나지 않았더라면...그때 확실하...,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99789,#@이름#들 데이트폭력 근절해주세요 특히 애인 아내 죽이는 #@이름#에게 사형을 주...,0,0,0,1,0,0,0,0,0,0,0,0
99795,"''한남'''들은 페미, ''메갈''' 이런것도 안하는데 말하는거 들으면 ''토악질...",2,0,0,2,3,0,0,0,0,0,0,0
99839,''헬조선''' ''새끼'''들은 인종차별이라고 왜 ''씨부리'''는지? 니네들도 ...,2,2,0,2,0,0,2,0,0,0,0,0
99905,시청자들이 ''갑'''인데 ''갑'''한테 ''그따구'''로 고지도 안하고 ''쳐맞...,0,3,0,2,0,0,0,0,0,0,0,0


### 특수 따옴표 패턴 내 단어 추출

- 특수 따옴표 패턴 내 단어는 특정 욕설을 의미함. 추출해서 Amazon Bedrock Guardrails의 사용자 지정단어에 사용
- 1글자의 경우 너무 많은 필터링이 되기 때문에 삭제
- Guardrails의 경우 현재 10,000개의 필터를 제공, 50kb이하, 한개당 최대 3 phrase
- Guardrails의 욕설필터를 최대한 막을 수 있는 사용자지정단어를 만듭니다.

In [21]:
# 특수 따옴표 패턴 내 단어 추출
block_list = create_block_list_from_special_quotes(violence_threats, '문장')

#1글자 단어 제거 함수
filtered_block_list = remove_single_char_words(block_list)

# 결과 확인
print("\n최종 차단 단어/구문 목록:")
print(filtered_block_list)

# CSV 파일로 저장
filtered_block_list.to_csv('guardrail_filtered_block_list.csv', index=False)
print("\n필터링된 차단 목록이 'guardrail_filtered_block_list.csv'로 저장되었습니다.")

제거된 1글자 단어 (380개):

필터링 전: 7252개 단어
필터링 후: 6872개 단어
제거된 단어 수: 380개

최종 차단 단어/구문 목록:
     blocked_words
1          #@이름#정권
2               ??
3              강아지
4             깽깽이 
5               나불
...            ...
7247         흙수저노예
7248           흡혈기
7249            희생
7250          히로시마
7251            힘든

[6872 rows x 1 columns]

필터링된 차단 목록이 'guardrail_filtered_block_list.csv'로 저장되었습니다.


### 특수 따옴표 패턴의 한국어 욕설중 구를 제외한 명사만을 추출
- 다양하게 변경하세요.

In [22]:
from konlpy.tag import Okt, Mecab
import pandas as pd

def extract_korean_nouns(block_list_df, min_length=2):
    """
    차단 목록에서 한국어 명사만 추출하는 함수
    
    Parameters:
    block_list_df (DataFrame): 차단 단어 목록이 있는 DataFrame
    min_length (int): 추출할 명사의 최소 길이 (기본값: 2)
    
    Returns:
    DataFrame: 한국어 명사만 포함된 차단 단어 목록 DataFrame
    """
    try:
        # Okt 형태소 분석기 초기화 (또는 try-except로 Mecab 시도)
        okt = Okt()
    except:
        print("Okt 초기화 실패, Mecab을 시도합니다...")
        try:
            mecab = Mecab()
        except:
            print("주의: KoNLPy 형태소 분석기 초기화 실패. 명사 추출이 제한적일 수 있습니다.")
            # 형태소 분석기 없이 기본적인 필터링만 수행
            return block_list_df

    # 결과를 저장할 리스트
    extracted_nouns = []
    skipped_words = []
    
    # 각 단어에서 명사 추출
    for word in block_list_df['blocked_words']:
        if not isinstance(word, str):
            continue
            
        try:
            # Okt로 명사 추출 시도
            nouns = okt.nouns(word)
            
            # 명사가 없거나 모두 짧은 경우, 원래 단어 유지 (욕설/비속어는 명사로 인식되지 않을 수 있음)
            if not nouns or all(len(noun) < min_length for noun in nouns):
                if len(word.replace(" ", "")) >= min_length:
                    extracted_nouns.append(word)
                else:
                    skipped_words.append(word)
            else:
                # min_length 이상 길이의 명사만 추가
                valid_nouns = [noun for noun in nouns if len(noun) >= min_length]
                extracted_nouns.extend(valid_nouns)
                
                # 추출된 명사가 없으면 원래 단어 추가 고려
                if not valid_nouns and len(word.replace(" ", "")) >= min_length:
                    extracted_nouns.append(word)
        except Exception as e:
            print(f"'{word}' 처리 중 오류 발생: {e}")
            # 오류 발생 시 원래 단어 유지
            if len(word.replace(" ", "")) >= min_length:
                extracted_nouns.append(word)
    
    # 중복 제거 및 정렬
    extracted_nouns = sorted(list(set(extracted_nouns)))
    
    # 결과 DataFrame 생성
    result_df = pd.DataFrame(extracted_nouns, columns=['blocked_words'])
    
    # 통계 출력
    print(f"원본 단어 수: {len(block_list_df)}")
    print(f"추출된 명사 수: {len(result_df)}")
    print(f"건너뛴 단어 수: {len(skipped_words)}")
    
    if skipped_words:
        print("\n건너뛴 단어 목록:")
        for word in skipped_words:
            print(f"- '{word}'")
    
    return result_df

In [23]:
# 3. 한국어 명사 추출
noun_block_list = extract_korean_nouns(filtered_block_list, min_length=2)

# 결과 확인
print("\n최종 명사 기반 차단 단어/구문 목록:")
print(noun_block_list)

# CSV 파일로 저장
noun_block_list.to_csv('guardrail_noun_block_list.csv', index=False)
print("\n명사 기반 차단 목록이 'guardrail_noun_block_list.csv'로 저장되었습니다.")

원본 단어 수: 6872
추출된 명사 수: 5541
건너뛴 단어 수: 0

최종 명사 기반 차단 단어/구문 목록:
     blocked_words
0               ??
1             깽깽이 
2            덮어씌우기
3               문가
4               미쳐
...            ...
5536            흡혈
5537            희롱
5538            희생
5539          히로시마
5540            힘든

[5541 rows x 1 columns]

명사 기반 차단 목록이 'guardrail_noun_block_list.csv'로 저장되었습니다.
